# DIABETES LEARNING



In [ ]:
# !pip install matplotlib==3.3.4
# !pip install pycocotools
# !pip install plotly
# !pip install ipyplot
# !pip install opencv-python

In [1]:
import sys; print('Python:',sys.version)
import torch; print('Pytorch:',torch.__version__)
import fastai; print('Fastai:',fastai.__version__)

Python: 3.8.10 (default, Jun  4 2021, 15:09:15) 
[GCC 7.5.0]
Pytorch: 1.10.2+rocm4.2
Fastai: 2.5.3


In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.basics import *
from fastai.callback.all import *
from fastai.vision.all import *
import torch
from PIL import Image
from pycocotools.coco import COCO


## Data Inspection

In [4]:
train_folder = "/mnt/c/Users/bruno/Documents/diabetesLearning/foodDatabase/aiCrowd/train-v0.4/train/"
val_folder = "/mnt/c/Users/bruno/Documents/diabetesLearning/foodDatabase/aiCrowd/train-v0.4/val/"
train_coco = COCO(train_folder + "annotations.json")
val_coco = COCO(val_folder + "annotations.json")

coco_dataset = train_coco
coco_dataset_folder = train_folder

loading annotations into memory...
Done (t=3.02s)
creating index...
index created!
loading annotations into memory...
Done (t=0.16s)
creating index...
index created!


In [5]:
# Reading all classes

category_ids = coco_dataset.loadCats(coco_dataset.getCatIds())
# category_names = [_["name_readable"] for _ in category_ids]
category_names_dict = { cat["id"] : cat["name"] for cat in category_ids}

category_ids[0]

{'id': 2578,
 'name': 'water',
 'name_readable': 'Water',
 'supercategory': 'food'}

In [6]:
drinks = ["water", "tea", "coffe", "wine", "espresso","oil", "vinegar", "yogurt", "cappuccino", "caffeine", "milk", "juice", "drink", "boisson", "beer"]

foundDrink = []
for id, category in category_names_dict.items():
    categoryIsDrink = False
    for drink in drinks:
        if drink in category:
            categoryIsDrink = True
    if categoryIsDrink:
        foundDrink.append(category)

print(", ".join(foundDrink))
print(len(foundDrink))

remove = ['water-mineral', 'wine-white', 'juice-orange', 'boisson-au-glucose-50g',
 'oil-vinegar-salad-dressing', 'tea-black', 'water', 'wine-rosa-c',
  'tea-peppermint', 'ristretto-with-caffeine', 'kefir-drink', 'espresso-with-caffeine', 'tea-green', 
   'syrup-diluted-ready-to-drink',
    'cappuccino', 'wine-red', 'milk', 'coffee-with-caffeine',
     'ice-tea', 'white-coffee-with-caffeine', 'watermelon-fresh', 'herbal-tea', 'juice-apple', 'water-with-lemon-juice', 'tea-verveine', 'tea', 'beer']

filtered_category_names_dict = {}
for id, category in category_names_dict.items():
    if category not in remove:
        filtered_category_names_dict[id] = category

print(", ".join(filtered_category_names_dict.values()))
print(len(filtered_category_names_dict))

water, tea-green, white-coffee-with-caffeine, tea-black, espresso-with-caffeine, wine-white, oil-vinegar-salad-dressing, wine-red, coffee-with-caffeine, potatoes-steamed, boisson-au-glucose-50g, beer, tea-peppermint, green-bean-steamed-without-addition-of-salt, cappuccino, ristretto-with-caffeine, chicken-curry-cream-coconut-milk-curry-spices-paste, mashed-potatoes-prepared-with-full-fat-milk-with-butter, beetroot-steamed-without-addition-of-salt, milk, tea, juice-orange, beef-sirloin-steak, water-mineral, juice-apple, ice-tea, syrup-diluted-ready-to-drink, milk-chocolate, herbal-tea, water-with-lemon-juice, tea-verveine, kefir-drink, wine-rosa-c, watermelon-fresh
34
pear, egg, grapes, butter, bread-white, jam, bread-whole-wheat, apple, mixed-salad-chopped-without-sauce, cheese, tomato-sauce, pasta-spaghetti, carrot, onion, beef-cut-into-stripes-only-meat, rice-noodles-vermicelli, salad-leaf-salad-green, bread-grain, banana, mixed-vegetables, bread-wholemeal, savoury-puff-pastry, dried

In [6]:
filtered_category_names = list(filtered_category_names_dict.values())
catIds = coco_dataset.getCatIds(catNms=filtered_category_names) #specify which categories you want to retrieve
# catIds = coco_dataset.getCatIds() #get all categories

In [7]:
# generate classesIds for the ground truth mask

classesId = {name: i+1 for i, name in enumerate(filtered_category_names_dict.values())}

In [8]:
# Getting all categories with respective to their total images

no_images_per_category = {}
allImageIds = []
for n, i in enumerate(catIds):
  imgIds = coco_dataset.getImgIds(catIds=i)
  label = filtered_category_names[n]
  no_images_per_category[label] = len(imgIds)
  allImageIds += imgIds

allImageIds = list(set(allImageIds)) # remove duplicates
print(len(allImageIds))

no_images_per_category

18333


{'pear': 151,
 'egg': 626,
 'grapes': 94,
 'butter': 1008,
 'bread-white': 1273,
 'jam': 502,
 'bread-whole-wheat': 223,
 'apple': 504,
 'mixed-salad-chopped-without-sauce': 374,
 'cheese': 404,
 'tomato-sauce': 289,
 'pasta-spaghetti': 255,
 'carrot': 893,
 'onion': 144,
 'beef-cut-into-stripes-only-meat': 41,
 'rice-noodles-vermicelli': 50,
 'salad-leaf-salad-green': 1189,
 'bread-grain': 102,
 'banana': 412,
 'mixed-vegetables': 624,
 'bread-wholemeal': 901,
 'savoury-puff-pastry': 49,
 'dried-meat': 140,
 'fresh-cheese': 52,
 'red-radish': 138,
 'hard-cheese': 315,
 'ham-raw': 161,
 'bread-fruit': 48,
 'tomato': 1069,
 'cauliflower': 116,
 'potato-gnocchi': 58,
 'sauce-cream': 131,
 'pasta-linguini-parpadelle-tagliatelle': 75,
 'french-beans': 160,
 'almonds': 159,
 'dark-chocolate': 213,
 'mandarine': 138,
 'semi-hard-cheese': 101,
 'croissant': 144,
 'sushi': 56,
 'berries': 64,
 'biscuits': 134,
 'thickened-cream-35': 81,
 'corn': 130,
 'celeriac': 53,
 'alfa-sprouts': 40,
 'chi

In [9]:
import functools
import cv2
import numpy as np

def isNoisyAnnotation(masks):
  
  #### check if masks have high iou between each other
  intersectionThreshold = 0.3
  for mask1 in masks: 
    for mask2 in masks: # compare every mask with each other 
      if not np.array_equal(mask1,mask2): 
        intersectionPixels = np.sum(np.logical_and(mask1, mask2))
        smallestMask = min(np.sum(mask1),np.sum(mask2))
        percentualIntersection = intersectionPixels / smallestMask
        # print(percentualIntersection)
        if percentualIntersection > intersectionThreshold:
          print("Masks with high intersection")
          return True

  #### check if exists multiple contours in one mask and if these regions are too small
  areaThreshold = 0.01
  maxContours = 1

  for mask in masks:    
    mask = mask * 255 # transform binary mask from (0,1) format to (0,255)

    # fig = plt.figure(figsize=(9,9))
    # ax1 = fig.add_subplot(1,2,1)
    # ax1.imshow(mask)
    # plt.show()

    contours, hierarchy = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    numContours = len(contours)
  
    for contour in contours:
      area = cv2.contourArea(contour)
      if area == 0.0:
        print("Found zero area")
        numContours -= 1
        continue

      totalPixels = mask.shape[0] * mask.shape[1]
      # print(area/totalPixels)
      if area/totalPixels < areaThreshold:
        print("Very Small contour")
        return True

    # print(numContours)
    if numContours == 0:
      print("No contours")
      return True    

    if numContours > maxContours:
      print("Multiple contours")
      return True

  return False

def concatMasks(mask1, mask2):
  bothNotZero = np.logical_and(mask1, mask2)
  mask2[bothNotZero] = 0
  return mask1+mask2

def concatenateAllMasks(masks, ids, checkNoisyAnnotation=True):
    if checkNoisyAnnotation and isNoisyAnnotation(masks):
      return None

    masksWithLabel = []
    for mask, id in zip(masks, ids): 
        masksWithLabel.append(mask*id)
        
    concatenatedMasks = functools.reduce(concatMasks, masksWithLabel)
    return concatenatedMasks

In [10]:
import os

start = 0
for idx, _ in enumerate(allImageIds):
    idx = idx + start
    img_coco = coco_dataset.loadImgs(ids=allImageIds[idx])[0]
    img_path = coco_dataset_folder + "images/" + img_coco['file_name']

    annIds = coco_dataset.getAnnIds(imgIds=img_coco['id'])
    anns = coco_dataset.loadAnns(annIds)

    try:
        classes = []
        masks = []
        for ann in anns:
            mask = coco_dataset.annToMask(ann)
            masks.append(mask)
            classes.append(filtered_category_names_dict[ann["category_id"]])

    except:
        print("Skipping image that has removed class")
        continue

    maskIds = [classesId[className] for className in classes]
    # print(", ".join(classes))
    # print(maskIds)
    
    # fig = plt.figure(figsize=(9,9))
    # img = Image.open(img_path)
    # arr = np.asarray(img)
    # ax1 = fig.add_subplot(1,2,1)
    # ax1.imshow(arr)
    # coco_dataset.showAnns(anns) # optional

    finalMask = concatenateAllMasks(masks,maskIds)
    if finalMask is None:
        print("Skipping noisy image")
        # plt.show()
        # print("-------------------------------------")
        continue

    # ax1 = fig.add_subplot(1,2,2)
    # ax1.imshow(finalMask)
    # plt.show()
    # print("-------------------------------------")

    os.rename(img_path, coco_dataset_folder + "done/" + img_coco['file_name']) # move original file
    im = Image.fromarray(finalMask)
    filename = img_coco['file_name'].split(".")[0]
    im.save(coco_dataset_folder + "gt/" + filename + "GT.png") # save mask




Very Small contour
Skipping noisy image
Found zero area
Masks with high intersection
Skipping noisy image
Masks with high intersection
Skipping noisy image
Masks with high intersection
Skipping noisy image
Masks with high intersection
Skipping noisy image
Found zero area
Found zero area
Masks with high intersection
Skipping noisy image
Masks with high intersection
Skipping noisy image
Masks with high intersection
Skipping noisy image
Masks with high intersection
Skipping noisy image
Multiple contours
Skipping noisy image
Multiple contours
Skipping noisy image
Skipping image that has removed class
Masks with high intersection
Skipping noisy image
Skipping image that has removed class
Masks with high intersection
Skipping noisy image
Skipping image that has removed class
Masks with high intersection
Skipping noisy image
Masks with high intersection
Skipping noisy image
Very Small contour
Skipping noisy image
Masks with high intersection
Skipping noisy image
Masks with high intersection
S